In [1]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-19 10:25:55.766962: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 10:25:55.792110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 10:25:56.156443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [2]:
# 모델 및 토크나이저 로드
model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 맞춤법 교정 함수
def correct_text(input_text):
    input_encoding = tokenizer("맞춤법을 고쳐주세요: " + input_text, return_tensors="pt")
    input_ids = input_encoding.input_ids.to(device)
    attention_mask = input_encoding.attention_mask.to(device)
    
    output_encoding = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )
    output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)
    return output_text


/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via t

In [7]:
# CSV 파일 로드
df = pd.read_csv("/home/yjtech2/Desktop/yurim/LLM/Data/냄새_악취_포함.csv", encoding="cp949")

# 맞춤법 교정 적용
df["교정된_제목"] = df["제목"].apply(correct_text)
df = df.iloc[:, [0, 2]]

df


,제목,교정된_제목
0,악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다.,냄새나는 뮬이 계속 흘러내려 물을 오염시키고 있습니다.
1,마을 골 목 길에 퇴비를 두어 악취 발생이 심하니 이동 조치해 주세요.,마을 골 목 길에 퇴비를 두어 냄새 발생이 심하니 이동 조치해 주세요.
2,노후된 모정 보수 요청ㅓ,노후된 모정 보수 요청.
3,인도에 패기물 컨테이너 적치로 통행 불편,인도에 패기물 컨테이너 적치로 통행 불편.
4,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다.
...,...,...
111,복합악취의 수인한도는?,복합 악취의 수인한도는?.
112,악취배출량 단위 및 산정방법은?,냄새 배출량 단위 및 산정 방법은?.
113,단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁조정대...,단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁 조정...
114,"악취배출사업장 기술지원 및 공공환경시설 악취기술진단, 악취물질 측정·분석 관련 질의응답",냄새 배출 사업장 기술 지원 및 공공 환경 시설 악취 기술 진단 및 냄새 물질 측정...


In [ ]:
import json
import torch
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    AutoModelForTokenClassification, 
    AutoTokenizer
)
from tqdm import tqdm
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

class TextCorrector:
    def __init__(self):
        # 띄어쓰기 모델 초기화
        self.spacing_tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
        self.spacing_model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")
        
        # 맞춤법 모델 초기화
        self.spelling_tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')
        self.spelling_model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
        
        # GPU 설정
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.spacing_model = self.spacing_model.to(self.device)
        self.spelling_model = self.spelling_model.to(self.device)
        
        # 띄어쓰기 레이블
        self.spacing_labels = ["UNK", "PAD", "O", "B", "I", "E", "S"]

    # 띄어쓰기 교정
    def correct_spacing(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 512 - 2
            org_text = text.replace(" ", "")
            results = []
            
            for start_idx in range(0, len(org_text), max_length):
                chunk = org_text[start_idx:start_idx + max_length]
                
                # 토큰화
                tokens = [self.spacing_tokenizer.cls_token_id]
                tokens.extend(sum([self.spacing_tokenizer.encode(char)[1:-1] for char in chunk], []))
                tokens.append(self.spacing_tokenizer.sep_token_id)
                
                # 예측
                with torch.no_grad():
                    inputs = torch.tensor([tokens]).to(self.device)
                    outputs = self.spacing_model(inputs)
                    predictions = torch.argmax(outputs.logits, dim=-1)
                
                # 결과 생성
                result = ""
                for idx, pred in enumerate(predictions.squeeze()[1:-1]):
                    if idx >= len(chunk):
                        break
                    result += chunk[idx]
                    if self.spacing_labels[pred] in ["E", "S"]:
                        result += " "
                        
                results.append(result.strip())
            
            return " ".join(results).strip()
            
        except Exception as e:
            print(f"Spacing error: {str(e)}")
            return text

    # 맞춤법 교정
    def correct_spelling(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 128 - 2
            chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
            results = []
            
            for chunk in chunks:
                inputs = self.spelling_tokenizer(
                    "맞춤법을 고쳐주세요: " + chunk,
                    return_tensors="pt",
                    truncation=True,
                    max_length=max_length
                ).to(self.device)
                
                outputs = self.spelling_model.generate(
                    **inputs,
                    max_length=max_length,
                    num_beams=5,
                    early_stopping=True
                )
                
                result = self.spelling_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                if result:
                    results.append(result)
            
            return " ".join(results).strip() or ""
            
        except Exception as e:
            print(f"Spelling error: {str(e)}")
            return ""

    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        return re.sub('[^\w\s]', '', text)

def load_data(file_path):

    
    df = pd.read_excel(file_path)
    
    print(df.info())
    
    return df

def process_dataset(file_path):
    # 데이터 로드 및 전처리기 초기화
    print('데이터 로드')
    df = load_data(file_path)
    
    corrector = TextCorrector()
    
    # 진행바 설정
    tqdm.pandas(desc="Processing corrections")
    
    # 띄어쓰기 교정
    print("Applying spacing correction...")
    df['spacing_res_sentence'] = df['text'].progress_apply(corrector.correct_spacing)
    
    # 맞춤법 교정
    print("Applying spelling correction...")
    df['spelling_res_sentence'] = df['spacing_res_sentence'].progress_apply(corrector.correct_spelling)
    
    # 텍스트 클리닝
    df['spelling_res_sentence'] = df['spelling_res_sentence'].apply(corrector.clean_text)
    return df

if __name__ == "__main__":
    file_path = '/home/yjtech2/Desktop/yurim/LLM/Data/text_test_data.xlsx'
    result_df = process_dataset(file_path)
    
    print("\n=== Results ===")
    result_df

데이터 로드
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes
None
Applying spacing correction...


Processing corrections: 100%|██████████| 10/10 [00:00<00:00, 301.70it/s]


Applying spelling correction...


Processing corrections: 100%|██████████| 10/10 [00:01<00:00,  9.73it/s]


=== Results ===


In [19]:
result_df = result_df[['text', 'spacing_res_sentence', 'spelling_res_sentence']]

result_df

,text,spacing_res_sentence,spelling_res_sentence
0,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄새가 나요
1,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음 \n바람 이불 면골 목안쪽...,전라북도 순창군 ᄋᄋ 면 마을 꼴 목길에 퇴비를 가져다 놓음 바람 이불 면골 목 안...
2,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다.\n발리조치해 주시기...,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다. \n발 리조치 해주...,인도에 폐기물 컨테이너가 있어 통행 불편 및 냄새로 불편합니다
3,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다
4,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄새 신고
5,층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요?\n원룸형 아파트 인데요...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요? \n원룸 형아파트인...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요
6,건물 옆 주차장에 길고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다
7,공공시설/ 길거리 흡연자가 녀무 많습니다,공공시설 / 길거리 흡연자가 녀무 많습니다,공공시설 길거리 흡연자가 너무 많습니다
8,벌래와 모기 하수구냄새가 납니다,벌래와 모기 하수구 냄새가 납니다,벌래와 모기 하수구 냄새가 납니다
9,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다


In [17]:
result_df.iloc[5]

text                     층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요?\n원룸형 아파트 인데요...
spacing_res_sentence     층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요? \n원룸 형아파트인...
spelling_res_sentence    층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요...
Name: 5, dtype: object

In [28]:
import json
import torch
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    AutoModelForTokenClassification, 
    AutoTokenizer
)
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from tqdm import tqdm
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

class TextCorrector:
    def __init__(self):
        # 띄어쓰기 모델 초기화
        self.spacing_tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
        self.spacing_model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")
        
        # 맞춤법 모델 초기화
        self.spelling_tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")
        self.spelling_model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
        
        # GPU 설정
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.spacing_model = self.spacing_model.to(self.device)
        self.spelling_model = self.spelling_model.to(self.device)
        
        # 띄어쓰기 레이블
        self.spacing_labels = ["UNK", "PAD", "O", "B", "I", "E", "S"]

    # 띄어쓰기 교정
    def correct_spacing(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 512 - 2
            org_text = text.replace(" ", "")
            results = []
            
            for start_idx in range(0, len(org_text), max_length):
                chunk = org_text[start_idx:start_idx + max_length]
                
                # 토큰화
                tokens = [self.spacing_tokenizer.cls_token_id]
                tokens.extend(sum([self.spacing_tokenizer.encode(char)[1:-1] for char in chunk], []))
                tokens.append(self.spacing_tokenizer.sep_token_id)
                
                # 예측
                with torch.no_grad():
                    inputs = torch.tensor([tokens]).to(self.device)
                    outputs = self.spacing_model(inputs)
                    predictions = torch.argmax(outputs.logits, dim=-1)
                
                # 결과 생성
                result = ""
                for idx, pred in enumerate(predictions.squeeze()[1:-1]):
                    if idx >= len(chunk):
                        break
                    result += chunk[idx]
                    if self.spacing_labels[pred] in ["E", "S"]:
                        result += " "
                        
                results.append(result.strip())
            
            return " ".join(results).strip()
            
        except Exception as e:
            print(f"Spacing error: {str(e)}")
            return text

    # 맞춤법 교정
    def correct_spelling(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 128 - 2
            chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
            results = []
            
            for chunk in chunks:
                inputs = self.spelling_tokenizer(
                    "맞춤법을 고쳐주세요: " + chunk,
                    return_tensors="pt",
                    truncation=True,
                    max_length=max_length
                ).to(self.device)
                
                outputs = self.spelling_model.generate(
                    **inputs,
                    max_length=max_length,
                    num_beams=5,
                    early_stopping=True
                )
                
                result = self.spelling_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                if result:
                    results.append(result)
            
            return " ".join(results).strip() or ""
            
        except Exception as e:
            print(f"Spelling error: {str(e)}")
            return ""

    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        return re.sub('[^\w\s]', '', text)

def load_data(file_path):

    
    df = pd.read_excel(file_path)
    
    print(df.info())
    
    return df

def process_dataset(file_path):
    # 데이터 로드 및 전처리기 초기화
    print('데이터 로드')
    df = load_data(file_path)
    
    corrector = TextCorrector()
    
    # 진행바 설정
    tqdm.pandas(desc="Processing corrections")
    
    # 띄어쓰기 교정
    print("Applying spacing correction...")
    df['spacing_res_sentence'] = df['text'].progress_apply(corrector.correct_spacing)
    
    # 맞춤법 교정
    print("Applying spelling correction...")
    df['spelling_res_sentence'] = df['spacing_res_sentence'].progress_apply(corrector.correct_spelling)
    
    # 텍스트 클리닝
    df['spelling_res_sentence'] = df['spelling_res_sentence'].apply(corrector.clean_text)
    return df

if __name__ == "__main__":
    file_path = '/home/yjtech2/Desktop/yurim/LLM/Data/text_test_data.xlsx'
    result_df = process_dataset(file_path)
    
    print("\n=== Results ===")
    result_df

데이터 로드
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes
None


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Applying spacing correction...


Processing corrections: 100%|██████████| 10/10 [00:00<00:00, 311.09it/s]


Applying spelling correction...


Processing corrections: 100%|██████████| 10/10 [00:04<00:00,  2.39it/s]


=== Results ===


In [29]:
result_df = result_df[['text', 'spacing_res_sentence', 'spelling_res_sentence']]

result_df

,text,spacing_res_sentence,spelling_res_sentence
0,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄세가 나요,맞춤법을 고쳐주세요 음식물 쓰레기 냄세가 나요 음식물 쓰레기 냄세가 나요\n 음식물...
1,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음 \n바람 이불 면골 목안쪽...,맞춤법을 고쳐주세요 전라북도 순창군 ᄋᄋ면 마을 꼴목길에 퇴비를 가져다 놓음 \n바...
2,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다.\n발리조치해 주시기...,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다. \n발 리조치 해주...,맞춤법을 고쳐주세요 인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다 ...
3,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,맞춤법을 고쳐주세요 이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주...
4,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄세 신고,맞춤법을 고쳐주세요 동국대 신공학관 근처 하수 처리 냄세 신고하세요 동국대 신공학관...
5,층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요?\n원룸형 아파트 인데요...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요? \n원룸 형아파트인...,맞춤법을 고쳐주세요 층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 ...
6,건물 옆 주차장에 길고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다,맞춤법을 고쳐주세요 건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다\n고양들이...
7,공공시설/ 길거리 흡연자가 녀무 많습니다,공공시설 / 길거리 흡연자가 녀무 많습니다,맞춤법을 고쳐주세요 공공시설 길거리 흡연자가 녀무 많습니다
8,벌래와 모기 하수구냄새가 납니다,벌래와 모기 하수구 냄새가 납니다,맞춤법을 고쳐주세요 벌래와 모기 하수구 냄새가 납니다\n벌래와 모기 하수구 냄새가 ...
9,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다,맞춤법을 고쳐주세요 바람이 동서족에서 불고 불쾌한 냄새가 납니다\n바람이 불고 불쾌...


In [32]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re

# 모델 및 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def split_sentences(text):
    # 문장 단위로 분리
    sentences = re.split(r'(?<=[.!?]) +', text.strip())
    return sentences

def generate_text(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    
    # 텍스트 생성
    outputs = model.generate(
        input_ids=input_ids,
        max_length=128,
        num_beams=5,
        repetition_penalty=2.0,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def correct_text(input_text):
    sentences = split_sentences(input_text)
    corrected_sentences = [generate_text(sentence) for sentence in sentences]
    return " ".join(corrected_sentences)

# 예제 입력
input_text = "오늘 날씨가 좋읍니다."
corrected_text = correct_text(input_text)
print("=== 입력 ===")
print(input_text)
print("\n=== 교정된 출력 ===")
print(corrected_text)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


=== 입력 ===
오늘 날씨가 좋읍니다.

=== 교정된 출력 ===
오늘 날씨가 좋읍니다. 고맙습니다."
"그런데 이게 웬일입니까?"
"아니오. 그건 그렇고요."
"어떻게 된 일입니까?"
"이제야 알겠습니다."
"무슨 말씀을 하시는 겁니까?"
"네, 저도 알고 있습니다."
"뭐라고요? 무슨 말씀을 하시는 겁니까?"
"저도 잘 모르겠습니다만."
"그렇지 않습니까?"
"왜 그러십니까?"
"우선 당신들한테 물어봐야 할 것 아닙니까?"
"당신들한테


In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")

def correct_text_et5(input_text):
    input_ids = tokenizer("맞춤법을 고쳐주세요: " + input_text, return_tensors="pt", truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

input_text = "안녕하세요. 포항시 오천읍에 거주하는 이강일입니다.  저는 지난 11월 13일 이후로 집에서 나오는 수돗물에서 고약한 냄새  가 나기 시작했습니다. 처음에는 아파트 물탱크의 문제가  있는 줄 알고 대수롭지 않게 여 겠는데 냄새가 계속 나서 걱정이  돼서 신고합니다. 현재 수돗물에서 흙냄새와 곰팡이  냄새가 나고 그래서 설거지나 세수 는 물론 물을 마실 수가 없습니다.생수로 대체해서 사용하고 있는데 이 문제로 매우 불편을 겪고 있습니다.  포항시 수돗물 원수의 40%를 공급 하는 경주 안개 때문에서 녹조  현상이 발생했다고 들었고 남조류 에서 발생한 지오스민이 냄새를  유발한다고 합니다. 이 문제에 대한 빠른 대응을 부탁  드립니다. 감사합니다."
corrected_text = correct_text_et5(input_text)
print(corrected_text)


2024-12-05 08:57:24.030602: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 08:57:24.054178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 08:57:24.464449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want 

안녕하세요? 포항 시 오천읍에 거주하는 이강일입니다.
